<a href="https://www.kaggle.com/code/sudhanshu2198/ipl-sports-analysis-using-sql?scriptVersionId=122325627" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
conn=sqlite3.connect("/kaggle/input/ipldatabase/database.sqlite")

# Indian Premier League

(<a href="https://ibb.co/k1H0jRm"><img src="https://i.ibb.co/gVjwN54/IPL.webp" alt="IPL" border="0" /></a>)

**Indian Premier League (IPL)** is a men's T20 franchise cricket league in India. It is annually contested by ten teams based on a round-robin group and knockout format, has teams in major Indian cities. The IPL has developed into the most lucrative and most popular outlet for the game of cricket. The brand value of the IPL in 2022 was **₹90,038 crore (US $ 11 billion)**.

The top four teams contest three play-off matches, with one losing team being given a second chance to reach the final, a wrinkle aimed at maximizing potential television revenue. The play-off portion of the tournament involves the four teams that finished at the top of the tables in a series of knockout games that allows one team that lost its first-round game a second chance to advance to the final match.

![Knock Out Format](https://staticg.sportskeeda.com/wp-content/uploads/2016/05/capture-1462822901-800.png)

### Database Schema

<a href="https://ibb.co/RyBt71B"><img src="https://i.ibb.co/HdxRX0x/Database-Schema.png" alt="Database-Schema" border="0"></a>

### Objective

The objective of this notebook is to do a rigorous analysis of Indian Prenier League Database, answering multiple comman questions that cricketting fans have like top scorer, best Captain, best death over Specialist and many more. This notebook can be extremely helpful to people who knows and applied SQL on dummy database but want to practise on a Real Database, along with structured framework( where every query builds on previous queries) and questions to help them start.The notebook contains 30+ Advance queries covering nested case,aggregrate,different types of join,groupby
and multiple table join.

[SQL Learning and Practising Resources Link](https://www.kaggle.com/discussions/questions-and-answers/350791)

[Kaggle Notebook Link](https://www.kaggle.com/code/sudhanshu2198/ipl-sports-analysis-using-sql)

[Database Schema Image Link](https://i.imgur.com/327NVKH.png)

### Bowling Statistics

In [3]:
#Bowlers who have bowled most deliveries

sql="""SELECT Player_Name,c.Country_Name,d.Bowling_skill,count(*) as Deliveries
       FROM Ball_by_Ball a
       join Player b
       on a.Bowler==b.Player_Id
       join Country c
       on b.Country_Name==c.Country_Id
       join Bowling_Style d
       on b.Bowling_skill==d.Bowling_Id
       group by Bowler
       order by Deliveries desc ;"""

bowling=pd.read_sql(sql, conn)
bowling.head().style.background_gradient(subset=["Deliveries"],cmap="YlOrBr_r",low=0.25)


,Player_Name,Country_Name,Bowling_skill,Deliveries
0,Harbhajan Singh,India,Right-arm offbreak,2742
1,P Kumar,India,Right-arm medium,2529
2,PP Chawla,India,Legbreak,2472
3,A Mishra,India,Legbreak,2466
4,SL Malinga,Sri Lanka,Right-arm fast,2407


In [4]:
#Highest Wicket takers in IPL
'''Runout, Retired hurt and Obstructing field are not counted as bowlers wicket''' 

sql=""" select Player_Name,count(Kind_Out) as Wickets,
        sum(case when Kind_Out==1 then 1 else 0 end) as caught,
        sum(case when Kind_Out==2 then 1 else 0 end) as bowled,
        sum(case when Kind_Out==4 then 1 else 0 end) as lbw,
        sum(case when Kind_Out==6 then 1 else 0 end) as stumped
        from Wicket_Taken a
        join Ball_by_Ball b
        on a.Match_Id=b.Match_Id 
        and a.Innings_No=b.Innings_No
        and a.Over_Id=b.Over_Id 
        and a.Ball_Id=b.Ball_Id
        join (select Player_Id,Player_Name,b.Country_Name, c.Bowling_skill
              from Player a
              join Country b
              on a.Country_Name=b.Country_Id
              join Bowling_Style c
              on a.Bowling_skill=c.Bowling_Id) c
        on b.Bowler=c.Player_Id
        where Kind_Out in (1,2,4,6,7,8)
        group by c.Player_Id
        order by Wickets desc; """

wickets=pd.read_sql(sql, conn)
subset=["Wickets","caught","bowled","lbw","stumped","caught and bowled"]
wickets.head().style.background_gradient(subset=["Wickets"],cmap="YlOrBr_r",low=0.25)

,Player_Name,Wickets,caught,bowled,lbw,stumped
0,SL Malinga,143,69,59,10,0
1,A Mishra,124,66,21,13,21
2,DJ Bravo,122,90,18,6,0
3,PP Chawla,120,56,33,14,13
4,Harbhajan Singh,119,61,25,8,14


In [5]:
# Highest wicket taken by a bowler in an IPL match

sql=""" select Player_Name,Country_Name,Bowling_skill,
        Wickets||"-"||runs as Best
        from( select a.Bowler,a.Match_Id,count(Kind_Out) as Wickets
              from Ball_by_Ball a
              join Wicket_Taken b
              on a.Match_Id=b.Match_Id 
              and a.Innings_No=b.Innings_No
              and a.Over_Id=b.Over_Id 
              and a.Ball_Id=b.Ball_Id
              where Kind_Out in (1,2,4,6,7,8)
              group by a.Bowler,a.Match_Id) a
        join( select a.Bowler,a.Match_Id,
              sum(coalesce(Runs_Scored,0)+coalesce(Extra_Runs,0)) as runs
              from Ball_by_Ball a
              left join Batsman_Scored b 
              on a.Match_Id=b.Match_Id 
              and a.Innings_No=b.Innings_No
              and a.Over_Id=b.Over_Id 
              and a.Ball_Id=b.Ball_Id
              left join Extra_Runs c
              on a.Match_Id=c.Match_Id 
              and a.Innings_No=c.Innings_No
              and a.Over_Id=c.Over_Id 
              and a.Ball_Id=c.Ball_Id
              group by a.Bowler,a.Match_Id) b
              on a.Bowler=b.Bowler
              and a.Match_Id=b.Match_Id
        join( select Player_Id,Player_Name,b.Country_Name, c.Bowling_skill
              from Player a
              join Country b
              on a.Country_Name=b.Country_Id
              join Bowling_Style c
              on a.Bowling_skill=c.Bowling_Id) c
        on a.Bowler=c.Player_Id
        order by Wickets desc; """

highest_wicket=pd.read_sql(sql, conn)
highest_wicket.head()

,Player_Name,Country_Name,Bowling_skill,Best
0,Sohail Tanvir,Pakistan,Left-arm medium-fast,6-15
1,A Zampa,Australia,Legbreak googly,6-19
2,RA Jadeja,India,Slow left-arm orthodox,5-16
3,Harbhajan Singh,India,Right-arm offbreak,5-18
4,I Sharma,India,Right-arm fast-medium,5-13


In [6]:
# No of 5-wicket hauls by bowlers in an IPL 

sql=""" select Player_Name,Country_Name,Bowling_skill,
        count(*) as hauls
        from (select Player_Id,Player_Name,Country_Name,Bowling_skill,
              count(Kind_Out) as Wickets
              from Wicket_Taken a
              join Ball_by_Ball b
              on a.Match_Id=b.Match_Id 
              and a.Innings_No=b.Innings_No
              and a.Over_Id=b.Over_Id 
              and a.Ball_Id=b.Ball_Id
              join (select Player_Id,Player_Name,b.Country_Name, c.Bowling_skill
                    from Player a
                    join Country b
                    on a.Country_Name=b.Country_Id
                    join Bowling_Style c
                    on a.Bowling_skill=c.Bowling_Id) c
             on b.Bowler=c.Player_Id
             where Kind_Out in (1,2,4,6,7,8)
             group by c.Player_Id,a.Match_Id
             having Wickets >=5)
             group by Player_Id
             order by hauls desc; """

highest_wicket=pd.read_sql(sql, conn)
highest_wicket.head().style.background_gradient(subset=["hauls"],cmap="YlOrBr_r",low=0.25)

,Player_Name,Country_Name,Bowling_skill,hauls
0,JP Faulkner,Australia,Left-arm fast-medium,2
1,A Zampa,Australia,Legbreak googly,1
2,JD Unadkat,India,Left-arm medium,1
3,SP Narine,West Indies,Right-arm offbreak,1
4,SL Malinga,Sri Lanka,Right-arm fast,1


In [7]:
# Most Runs Conceded by a Bowler in an IPL Match

sql=""" select Player_Name,Country_Name,Bowling_skill,runs
        from( select a.Bowler,
              sum(coalesce(Runs_Scored,0)+coalesce(Extra_Runs,0)) as runs
              from Ball_by_Ball a
              left join Batsman_Scored b 
              on a.Match_Id=b.Match_Id 
              and a.Innings_No=b.Innings_No
              and a.Over_Id=b.Over_Id 
              and a.Ball_Id=b.Ball_Id
              left join Extra_Runs c
              on a.Match_Id=c.Match_Id 
              and a.Innings_No=c.Innings_No
              and a.Over_Id=c.Over_Id 
              and a.Ball_Id=c.Ball_Id
              group by a.Bowler,a.Match_Id) a
        join( select Player_Id,Player_Name,b.Country_Name, c.Bowling_skill
              from Player a
              join Country b
              on a.Country_Name=b.Country_Id
              join Bowling_Style c
              on a.Bowling_skill=c.Bowling_Id) b
        on a.Bowler=b.Player_Id
        order by runs desc; """

runs_concede_in_match=pd.read_sql(sql, conn)
runs_concede_in_match.head().style.background_gradient(subset=["runs"],cmap="YlOrBr_r",low=0.25)

,Player_Name,Country_Name,Bowling_skill,runs
0,I Sharma,India,Right-arm fast-medium,66
1,Sandeep Sharma,India,Right-arm medium,66
2,UT Yadav,India,Right-arm fast-medium,65
3,AB Dinda,India,Right-arm medium-fast,63
4,VR Aaron,India,Right-arm fast,63


In [8]:
# Highest Runs Concede in an IPL over by a bowler

sql=""" select Player_Name as Bowler,
        max(runs+extra) as Runs,Fours,Sixes,
        Extra,wides,noballs,legbyes
        from(select Match_Id,Innings_No,Over_Id,sum(Runs_Scored) as runs,
             sum(case when Runs_Scored==4 then 1 else 0 end) as Fours,
             sum(case when Runs_Scored==6 then 1 else 0 end) as Sixes
             from Batsman_Scored
             group by Match_Id, Innings_No,Over_Id) a
        join(select Match_Id,Innings_No,Over_Id,sum(Extra_Runs) as Extra,
             sum(case when Extra_Type_Id==1 then 1 else 0 end) as legbyes,
             sum(case when Extra_Type_Id==2 then 1 else 0 end) as wides,
             sum(case when Extra_Type_Id==4 then 1 else 0 end) as noballs
             from Extra_Runs
             group by Match_Id,Innings_No,Over_Id) b
        on a.Match_Id=b.Match_Id 
        and a.Innings_No=b.Innings_No
        and a.Over_Id=b.Over_Id
        join (select Match_Id,Innings_No,Over_Id,Player_Name,
              c.Bowling_skill,d.Country_Name
              from Ball_by_Ball a
              join Player b
              on a.Bowler=b.Player_Id
              join Bowling_Style c
              on b.Bowling_skill=c.Bowling_Id
              join Country d
              on b.Country_Name=d.Country_Id
              group by Match_Id,Innings_No,Over_id) c
         on a.Match_Id=c.Match_Id 
         and a.Innings_No=c.Innings_No
         and a.Over_Id=c.Over_Id
         group by a.Match_Id,a.Innings_No,a.Over_Id
         order by Runs desc; """

most_runs_in_over=pd.read_sql(sql, conn)
most_runs_in_over.head().style.background_gradient(subset=["Runs"],cmap="YlOrBr_r",low=0.25)

,Bowler,Runs,Fours,Sixes,Extra,wides,noballs,legbyes
0,P Parameswaran,37,3,4,1,0,1,0
1,RS Bopara,33,0,4,7,2,0,0
2,P Awana,33,5,2,1,0,1,0
3,DJ Bravo,30,2,3,1,0,1,0
4,RP Singh,28,2,2,2,0,2,0


In [9]:
# best economy bowler's in IPL

sql=""" select Player_Name,Country_Name,
        overs,runs,extras,economy
        from(select Bowler,sum(coalesce(Extra_Runs,0)+Runs_Scored) as runs,
             sum(coalesce(Extra_Runs,0)) as extras,
             count(*)/6 as overs,
             round(round(sum(coalesce(Extra_Runs,0)+Runs_Scored),2)/round(count(*)/6,2),2) as economy
             from Batsman_Scored a
             join Ball_by_Ball b
             on a.Match_Id=b.Match_Id
             and a.Innings_No=b.Innings_No
             and a.Over_Id=b.Over_Id
             and a.Ball_Id=b.Ball_Id
             left join Extra_Runs c
             on a.Match_Id=c.Match_Id
             and a.Innings_No=c.Innings_No
             and a.Over_Id=c.Over_Id
             and a.Ball_Id=c.Ball_Id
             group by Bowler
             having overs>=50) a
        join (select Player_Id,Player_Name,b.Bowling_skill,c.Country_Name
              from Player a
              join Bowling_Style b
              on a.Bowling_skill=b.Bowling_Id
              join Country c
              on a.Country_Name=c.Country_Id) b
        on a.Bowler=b.Player_Id 
        order by economy asc; """

best_economy=pd.read_sql(sql, conn)
display(best_economy.head(10).style.background_gradient(subset=["economy"],cmap="YlOrBr_r",low=0.25))

,Player_Name,Country_Name,overs,runs,extras,economy
0,SP Narine,West Indies,263,1657,52,6.300000
1,R Ashwin,India,383,2475,66,6.460000
2,SL Malinga,Sri Lanka,386,2534,127,6.560000
3,DW Steyn,South Africa,347,2287,78,6.590000
4,A Kumble,India,161,1063,36,6.600000
5,M Muralitharan,Sri Lanka,256,1709,67,6.680000
6,GD McGrath,Australia,54,362,10,6.700000
7,Mustafizur Rahman,Bangladesh,61,417,6,6.840000
8,DL Vettori,New Zealand,129,888,18,6.880000
9,RE van der Merwe,South Africa,73,503,17,6.890000


In [10]:
#Worst Economy Bowlers

worst_economy=best_economy.tail(10).iloc[::-1,:].reset_index(drop=True)
display(worst_economy.style.background_gradient(subset=["economy"],cmap="YlOrBr_r",low=0.25))

,Player_Name,Country_Name,overs,runs,extras,economy
0,JR Hopes,Australia,60,553,14,9.220000
1,Pankaj Singh,India,50,459,6,9.180000
2,R McLaren,South Africa,59,541,21,9.170000
3,DJ Hussey,Australia,53,479,12,9.040000
4,VRV Singh,India,60,542,11,9.030000
5,Anureet Singh,India,63,552,19,8.760000
6,AB Agarkar,India,131,1144,32,8.730000
7,NLTC Perera,Sri Lanka,117,1020,22,8.720000
8,Mohammed Shami,India,78,677,23,8.680000
9,VY Mahesh,India,58,502,24,8.660000


In [11]:
#Best Death overs Bowler's in Indian Premier League
#Wickets_rate= average no of wickets ball in an over

sql="""select Player_Name,Country_Name,
       Overs,Runs,Wickets,Economy,
       round(round(Wickets,2)/round(Overs,2),2) as Wicket_rate
       from( select Bowler,count(*) as Overs,sum(runs) as Runs,
             round(round(sum(runs),2)/round(count(*),2),2) as Economy
             from (select a.Bowler,a.Match_Id,
                   sum(coalesce(Runs_Scored,0)+coalesce(Extra_Runs,0)) as runs
                   from Ball_by_Ball a
                   left join Batsman_Scored b 
                   on a.Match_Id=b.Match_Id 
                   and a.Innings_No=b.Innings_No
                   and a.Over_Id=b.Over_Id 
                   and a.Ball_Id=b.Ball_Id
                   left join Extra_Runs c
                   on a.Match_Id=c.Match_Id 
                   and a.Innings_No=c.Innings_No
                   and a.Over_Id=c.Over_Id 
                   and a.Ball_Id=c.Ball_Id
                   where a.Over_Id in (16,17,18,19,20)
                   group by a.Bowler,a.Match_Id,a.Over_Id)
              group by Bowler) a 
        join( select Bowler,sum(Kind_Out) as Wickets
              from Ball_by_Ball a
              join Wicket_Taken b
              on a.Match_Id=b.Match_Id 
              and a.Innings_No=b.Innings_No
              and a.Over_Id=b.Over_Id 
              and a.Ball_Id=b.Ball_Id
              where a.Over_Id in (16,17,18,19,20) and
              Kind_Out in (1,2,4,6,7,8)
              group by a.Bowler) b
        on a.Bowler=b.Bowler
        join( select Player_Id,Player_Name,b.Country_Name, c.Bowling_skill
              from Player a
              join Country b
              on a.Country_Name=b.Country_Id
              join Bowling_Style c
              on a.Bowling_skill=c.Bowling_Id) c
      on a.Bowler=c.Player_Id
      where Overs>=50
      order by Economy,Wicket_rate desc; """

death_overs_specialist=pd.read_sql(sql, conn)
subset=["Economy","Wicket_rate"]
death_overs_specialist.head().style.background_gradient(subset=subset,cmap="YlOrBr_r",low=0.25)

,Player_Name,Country_Name,Overs,Runs,Wickets,Economy,Wicket_rate
0,SP Narine,West Indies,98,718,130,7.330000,1.330000
1,SL Malinga,Sri Lanka,160,1172,180,7.330000,1.130000
2,R Ashwin,India,61,479,43,7.850000,0.700000
3,DW Steyn,South Africa,101,806,80,7.980000,0.790000
4,PP Chawla,India,56,449,70,8.020000,1.250000


In [12]:
#Poor Death overs Bowler's in Indian Premier League
subset=["Economy","Wicket_rate"]
death_overs_poor=death_overs_specialist.tail().iloc[::-1,:].reset_index(drop=True)
death_overs_poor.style.background_gradient(subset=subset,cmap="YlOrBr_r",low=0.25)

,Player_Name,Country_Name,Overs,Runs,Wickets,Economy,Wicket_rate
0,I Sharma,India,56,611,16,10.910000,0.290000
1,AB Dinda,India,67,698,30,10.420000,0.450000
2,MM Patel,India,52,542,36,10.420000,0.690000
3,P Kumar,India,98,1019,50,10.400000,0.510000
4,JH Kallis,South Africa,60,617,38,10.280000,0.630000


In [13]:
#Best powerplay overs Bowler's in Indian Premier League
#Wickets_rate= average no of wickets ball in an over

sql="""select Player_Name,Country_Name,
       Overs,Runs,Wickets,Economy,
       round(round(Wickets,2)/round(Overs,2),2) as Wicket_rate
       from( select Bowler,count(*) as Overs,sum(runs) as Runs,
             round(round(sum(runs),2)/round(count(*),2),2) as Economy
             from (select a.Bowler,a.Match_Id,
                   sum(coalesce(Runs_Scored,0)+coalesce(Extra_Runs,0)) as runs
                   from Ball_by_Ball a
                   left join Batsman_Scored b 
                   on a.Match_Id=b.Match_Id 
                   and a.Innings_No=b.Innings_No
                   and a.Over_Id=b.Over_Id 
                   and a.Ball_Id=b.Ball_Id
                   left join Extra_Runs c
                   on a.Match_Id=c.Match_Id 
                   and a.Innings_No=c.Innings_No
                   and a.Over_Id=c.Over_Id 
                   and a.Ball_Id=c.Ball_Id
                   where a.Over_Id in (1,2,3,4,5,6)
                   group by a.Bowler,a.Match_Id,a.Over_Id)
              group by Bowler) a 
        join( select Bowler,sum(Kind_Out) as Wickets
              from Ball_by_Ball a
              join Wicket_Taken b
              on a.Match_Id=b.Match_Id 
              and a.Innings_No=b.Innings_No
              and a.Over_Id=b.Over_Id 
              and a.Ball_Id=b.Ball_Id
              where a.Over_Id in (1,2,3,4,5,6) and
              Kind_Out in (1,2,4,6,7,8)
              group by a.Bowler) b
        on a.Bowler=b.Bowler
        join( select Player_Id,Player_Name,b.Country_Name, c.Bowling_skill
              from Player a
              join Country b
              on a.Country_Name=b.Country_Id
              join Bowling_Style c
              on a.Bowling_skill=c.Bowling_Id) c
      on a.Bowler=c.Player_Id
      where Overs>=50
      order by Economy,Wicket_rate desc; """

powerplay_overs_specialist=pd.read_sql(sql, conn)
subset=["Economy","Wicket_rate"]
powerplay_overs_specialist.head().style.background_gradient(subset=subset,cmap="YlOrBr_r",low=0.25)

,Player_Name,Country_Name,Overs,Runs,Wickets,Economy,Wicket_rate
0,SP Narine,West Indies,64,354,26,5.530000,0.410000
1,SL Malinga,Sri Lanka,147,902,53,6.140000,0.360000
2,R Ashwin,India,117,729,77,6.230000,0.660000
3,B Kumar,India,151,945,78,6.260000,0.520000
4,DW Steyn,South Africa,177,1136,56,6.420000,0.320000


In [14]:
#Poor Powerplay overs Bowler's in Indian Premier League
subset=["Economy","Wicket_rate"]
powerplay_overs_poor=powerplay_overs_specialist.tail().iloc[::-1,:].reset_index(drop=True)
powerplay_overs_poor.style.background_gradient(subset=subset,cmap="YlOrBr_r",low=0.25)

,Player_Name,Country_Name,Overs,Runs,Wickets,Economy,Wicket_rate
0,MS Gony,India,80,705,30,8.810000,0.380000
1,JA Morkel,South Africa,150,1300,71,8.670000,0.470000
2,UT Yadav,India,98,776,30,7.920000,0.310000
3,CH Morris,South Africa,53,420,19,7.920000,0.360000
4,MJ McClenaghan,New Zealand,51,403,10,7.900000,0.200000


In [15]:
#Best Middle overs Bowler's in Indian Premier League
#Wickets_rate= average no of wickets ball in an over

sql="""select Player_Name,Country_Name,
       Overs,Runs,Wickets,Economy,
       round(round(Wickets,2)/round(Overs,2),2) as Wicket_rate
       from( select Bowler,count(*) as Overs,sum(runs) as Runs,
             round(round(sum(runs),2)/round(count(*),2),2) as Economy
             from (select a.Bowler,a.Match_Id,
                   sum(coalesce(Runs_Scored,0)+coalesce(Extra_Runs,0)) as runs
                   from Ball_by_Ball a
                   left join Batsman_Scored b 
                   on a.Match_Id=b.Match_Id 
                   and a.Innings_No=b.Innings_No
                   and a.Over_Id=b.Over_Id 
                   and a.Ball_Id=b.Ball_Id
                   left join Extra_Runs c
                   on a.Match_Id=c.Match_Id 
                   and a.Innings_No=c.Innings_No
                   and a.Over_Id=c.Over_Id 
                   and a.Ball_Id=c.Ball_Id
                   where a.Over_Id in (7,8,9,10,11,12,13,14,15)
                   group by a.Bowler,a.Match_Id,a.Over_Id)
              group by Bowler) a 
        join( select Bowler,sum(Kind_Out) as Wickets
              from Ball_by_Ball a
              join Wicket_Taken b
              on a.Match_Id=b.Match_Id 
              and a.Innings_No=b.Innings_No
              and a.Over_Id=b.Over_Id 
              and a.Ball_Id=b.Ball_Id
              where a.Over_Id in (1,2,3,4,5,6) and
              Kind_Out in (1,2,4,6,7,8)
              group by a.Bowler) b
        on a.Bowler=b.Bowler
        join( select Player_Id,Player_Name,b.Country_Name, c.Bowling_skill
              from Player a
              join Country b
              on a.Country_Name=b.Country_Id
              join Bowling_Style c
              on a.Bowling_skill=c.Bowling_Id) c
      on a.Bowler=c.Player_Id
      where Overs>=50
      order by Economy,Wicket_rate desc; """

middle_overs_specialist=pd.read_sql(sql, conn)
subset=["Economy","Wicket_rate"]
middle_overs_specialist.head().style.background_gradient(subset=subset,cmap="YlOrBr_r",low=0.25)

,Player_Name,Country_Name,Overs,Runs,Wickets,Economy,Wicket_rate
0,SP Narine,West Indies,102,601,26,5.890000,0.250000
1,MM Patel,India,60,356,53,5.930000,0.880000
2,RE van der Merwe,South Africa,52,322,7,6.190000,0.130000
3,J Botha,South Africa,66,425,16,6.440000,0.240000
4,Shakib Al Hasan,Bangladesh,79,515,21,6.520000,0.270000


In [16]:
#Best Bowlers in IPL 

sql=""" select Player_Name,Country_Name,
        Matches,Runs,Wickets,Economy,Best
        from(select Bowler,sum(coalesce(Extra_Runs,0)+Runs_Scored) as Runs,
             count(*)/6 as overs,
             round(round(sum(coalesce(Extra_Runs,0)+Runs_Scored),2)/round(count(*)/6,2),2) as Economy
             from Ball_by_Ball a
             left join Batsman_Scored b
             on a.Match_Id=b.Match_Id
             and a.Innings_No=b.Innings_No
             and a.Over_Id=b.Over_Id
             and a.Ball_Id=b.Ball_Id
             left join Extra_Runs c
             on a.Match_Id=c.Match_Id
             and a.Innings_No=c.Innings_No
             and a.Over_Id=c.Over_Id
             and a.Ball_Id=c.Ball_Id
             group by Bowler
             having overs>=50) a 
        join( select Bowler,count(Kind_Out) as Wickets
              from Ball_by_Ball a
              join Wicket_Taken b
              on a.Match_Id=b.Match_Id 
              and a.Innings_No=b.Innings_No
              and a.Over_Id=b.Over_Id 
              and a.Ball_Id=b.Ball_Id
              where Kind_Out in (1,2,4,6,7,8)
              group by Bowler) b
        on a.Bowler=b.Bowler
        join( select a.Bowler,max(Wickets)||"-"||runs as Best
              from( select a.Bowler,a.Match_Id,count(Kind_Out) as Wickets
                    from Ball_by_Ball a
                    join Wicket_Taken b
                    on a.Match_Id=b.Match_Id 
                    and a.Innings_No=b.Innings_No
                    and a.Over_Id=b.Over_Id 
                    and a.Ball_Id=b.Ball_Id
                    where Kind_Out in (1,2,4,6,7,8)
                    group by a.Bowler,a.Match_Id) a
              join( select a.Bowler,a.Match_Id,
                    sum(coalesce(Runs_Scored,0)+coalesce(Extra_Runs,0)) as runs
                    from Ball_by_Ball a
                    left join Batsman_Scored b 
                    on a.Match_Id=b.Match_Id 
                    and a.Innings_No=b.Innings_No
                    and a.Over_Id=b.Over_Id 
                    and a.Ball_Id=b.Ball_Id
                    left join Extra_Runs c
                    on a.Match_Id=c.Match_Id 
                    and a.Innings_No=c.Innings_No
                    and a.Over_Id=c.Over_Id 
                    and a.Ball_Id=c.Ball_Id
                    group by a.Bowler,a.Match_Id) b
              on a.Bowler=b.Bowler
              group by a.Bowler) c
        on a.Bowler=c.Bowler
        join( select Bowler,count (distinct Match_Id)as Matches
              from Ball_by_Ball
              group by Bowler) d
        on a.Bowler=d.Bowler
        join (select Player_Id,Player_Name,b.Bowling_skill,c.Country_Name
              from Player a
              join Bowling_Style b
              on a.Bowling_skill=b.Bowling_Id
              join Country c
              on a.Country_Name=c.Country_Id) e
        on a.Bowler=e.Player_Id
        order by Economy,Wickets; """

best_bowler=pd.read_sql(sql, conn)
subset=["Runs","Economy"]
best_bowler.head().style.background_gradient(subset=subset,cmap="YlOrBr_r",low=0.25)

,Player_Name,Country_Name,Matches,Runs,Wickets,Economy,Best
0,SP Narine,West Indies,66,1657,85,6.250000,5-11
1,R Ashwin,India,108,2475,100,6.300000,4-3
2,SL Malinga,Sri Lanka,98,2534,143,6.320000,5-6
3,DW Steyn,South Africa,90,2287,92,6.370000,3-6
4,M Muralitharan,Sri Lanka,66,1709,64,6.500000,4-8


In [17]:
#Worst Bowlers in IPL

subset=["Runs","Economy"]
worst_bowler=best_bowler.tail().iloc[::-1,:].reset_index(drop=True)
worst_bowler.style.background_gradient(subset=subset,cmap="YlOrBr_r",low=0.25)

,Player_Name,Country_Name,Matches,Runs,Wickets,Economy,Best
0,JR Hopes,Australia,20,553,14,9.070000,2-3
1,DJ Hussey,Australia,26,479,8,9.040000,2-2
2,Pankaj Singh,India,17,459,11,9.000000,2-12
3,A Mithun,India,16,463,7,8.900000,2-14
4,VRV Singh,India,18,542,12,8.890000,3-5


### Batting Statistics

In [18]:
#Batsmens who have faced most deliveries

sql="""SELECT Player_Name,c.Country_Name,d.Batting_hand,count(*) as Deliveries
       FROM Ball_by_Ball a
       join Player b
       on a.Striker==b.Player_Id
       join Country c
       on b.Country_Name==c.Country_Id
       join Batting_Style d
       on b.Batting_hand==d.Batting_Id
       group by Striker
       order by Deliveries desc ;"""

batters=pd.read_sql(sql, conn)
batters.head().style.background_gradient(subset=["Deliveries"],low=0.55)

,Player_Name,Country_Name,Batting_hand,Deliveries
0,V Kohli,India,Right-hand bat,3237
1,SK Raina,India,Left-hand bat,3059
2,G Gambhir,India,Left-hand bat,3028
3,RG Sharma,India,Right-hand bat,2996
4,RV Uthappa,India,Right-hand bat,2721


In [19]:
# Highest run scored by a batsman in an IPL 

sql=""" select Player_Name,Country_Name,Batting_hand,sum(Runs_Scored) as Runs
        from Batsman_Scored a
        join Ball_by_Ball b
        on a.Match_Id=b.Match_Id 
        and a.Innings_No=b.Innings_No
        and a.Over_Id=b.Over_Id 
        and a.Ball_Id=b.Ball_Id
        join (select Player_Id,Player_Name,b.Country_Name, c.Batting_hand
              from Player a
              join Country b
              on a.Country_Name=b.Country_Id
              join Batting_Style c
              on a.Batting_hand=c.Batting_Id) c
        on b.Striker=c.Player_Id
        group by c.Player_Id
        order by Runs desc; """

runs=pd.read_sql(sql, conn)
runs.head().style.background_gradient(subset=["Runs"],low=0.55)

,Player_Name,Country_Name,Batting_hand,Runs
0,SK Raina,India,Left-hand bat,4106
1,V Kohli,India,Right-hand bat,4105
2,RG Sharma,India,Right-hand bat,3874
3,G Gambhir,India,Left-hand bat,3634
4,CH Gayle,West Indies,Left-hand bat,3447


In [20]:
# Player who got dismissed at duck(0 score) highest no of times

sql=""" select Player_Name,Country_Name,Batting_hand,count(runs) as ducks
        from(select Striker,sum(Runs_Scored) as runs
             from (select Match_Id,Innings_No,Over_Id,Ball_Id,Striker
                   from Ball_by_Ball 
                   where Striker in (select Player_Out
                                   from Wicket_Taken))a
            join Batsman_Scored b
            on a.Match_Id=b.Match_Id 
            and a.Innings_No=b.Innings_No
            and a.Over_Id=b.Over_Id 
            and a.Ball_Id=b.Ball_Id
            group by Striker,a.Match_Id
            having runs==0) a
        join (select Player_Id,Player_Name,b.Country_Name, c.Batting_hand
              from Player a
              join Country b
              on a.Country_Name=b.Country_Id
              join Batting_Style c
              on a.Batting_hand=c.Batting_Id) b
        on a.Striker=b.Player_Id
        group by Player_Id
        order by ducks desc; """

ducks=pd.read_sql(sql, conn)
ducks.head().style.background_gradient(subset=["ducks"],low=0.55)

,Player_Name,Country_Name,Batting_hand,ducks
0,Harbhajan Singh,India,Right-hand bat,11
1,G Gambhir,India,Left-hand bat,11
2,R Sharma,India,Right-hand bat,10
3,MK Pandey,India,Right-hand bat,10
4,PP Chawla,India,Left-hand bat,10


In [21]:
# Highest run score by a batsman in an IPL match

sql=""" select Player_Name,Country_Name,runs as highest_score,balls,
        dots,fours,sixes
        from(select Striker,sum(Runs_Scored) as runs, count(Runs_Scored) as balls,
             sum(case when Runs_Scored==0 then 1 else 0 end) as dots,
             sum(case when Runs_Scored==4 then 1 else 0 end) as fours,
             sum(case when Runs_Scored==6 then 1 else 0 end) as sixes
             from Ball_by_Ball a
             join Batsman_Scored b
             on a.Match_Id=b.Match_Id 
             and a.Innings_No=b.Innings_No
             and a.Over_Id=b.Over_Id 
             and a.Ball_Id=b.Ball_Id
             group by Striker,a.Match_Id) a
        join (select Player_Id,Player_Name,b.Country_Name, c.Batting_hand
             from Player a
             join Country b
             on a.Country_Name=b.Country_Id
             join Batting_Style c
             on a.Batting_hand=c.Batting_Id) b
        on a.Striker=b.Player_Id
        order by highest_score desc; """

highest_score=pd.read_sql(sql, conn)
highest_score.head().style.background_gradient(subset=["highest_score"],low=0.55)

,Player_Name,Country_Name,highest_score,balls,dots,fours,sixes
0,CH Gayle,West Indies,175,66,15,13,17
1,BB McCullum,New Zealand,158,73,19,10,13
2,AB de Villiers,South Africa,133,59,10,19,4
3,AB de Villiers,South Africa,129,52,14,10,12
4,CH Gayle,West Indies,128,62,21,7,13


In [22]:
# No of fifties and centruies by a batsman in an IPL

sql=""" select Player_Name,Country_Name,
        sum(case when runs>=50 and runs<100 then 1 else 0 end) as fifties,
        sum(case when runs>=100 then 1 else 0 end) as centuries,
        max(runs) as highest_score
        from(select Striker,sum(Runs_Scored) as runs
             from Ball_by_Ball a
             join Batsman_Scored b
             on a.Match_Id=b.Match_Id 
             and a.Innings_No=b.Innings_No
             and a.Over_Id=b.Over_Id 
             and a.Ball_Id=b.Ball_Id
             group by Striker,a.Match_Id
             having runs>=50) a
        join (select Player_Id,Player_Name,b.Country_Name, c.Batting_hand
             from Player a
             join Country b
             on a.Country_Name=b.Country_Id
             join Batting_Style c
             on a.Batting_hand=c.Batting_Id) b
        on a.Striker=b.Player_Id
        group by Player_Id
        order by fifties+centuries desc; """

best_player=pd.read_sql(sql, conn)
best_player.head(10).style.background_gradient(subset=["fifties","centuries","highest_score"],low=0.55,axis=0)

,Player_Name,Country_Name,fifties,centuries,highest_score
0,DA Warner,Australia,32,2,109
1,G Gambhir,India,31,0,93
2,V Kohli,India,27,4,113
3,RG Sharma,India,29,1,109
4,SK Raina,India,28,1,100
5,CH Gayle,West Indies,21,5,175
6,AB de Villiers,South Africa,22,3,133
7,S Dhawan,India,25,0,95
8,AM Rahane,India,23,1,103
9,SE Marsh,Australia,18,1,115


In [23]:
# Power Hitters of IPL

sql=""" select Player_Name,Country_Name,boundaries,fours,sixes
        from(select Striker,
             sum(case when Runs_Scored==4 or Runs_Scored==6 then 1 else 0 end) as boundaries,
             sum(case when Runs_Scored==4 then 1 else 0 end) as fours,
             sum(case when Runs_Scored==6 then 1 else 0 end) as sixes
             from Ball_by_Ball a
            join Batsman_Scored b
            on a.Match_Id=b.Match_Id 
            and a.Innings_No=b.Innings_No
            and a.Over_Id=b.Over_Id 
            and a.Ball_Id=b.Ball_Id
            group by Striker) a
        join (select Player_Id,Player_Name,b.Country_Name, c.Batting_hand
              from Player a
              join Country b
              on a.Country_Name=b.Country_Id
              join Batting_Style c
              on a.Batting_hand=c.Batting_Id) b
        on a.Striker=b.Player_Id
        order by sixes desc; """

power_hitters=pd.read_sql(sql, conn)
power_hitters.head().style.background_gradient(subset=["boundaries","sixes","fours"],low=0.55,axis=0)

,Player_Name,Country_Name,boundaries,fours,sixes
0,CH Gayle,West Indies,533,281,252
1,RG Sharma,India,487,323,164
2,SK Raina,India,521,360,161
3,V Kohli,India,508,360,148
4,YK Pathan,India,374,231,143


In [24]:
# Batsman's with Highest strike rate and batting_average in IPL

sql="""select Player_Name as Player,Country_Name
       Matches,Runs,
       round(round(Runs,2)/round(dismissals,2),2) as Batting_Avg,Strike_Rate
       from(select Striker,sum(coalesce(Runs_Scored,0)) as Runs,
             100*round(round(sum(coalesce(Runs_Scored,0)),2)/round(count(*),2),4) as Strike_Rate
             from Ball_by_Ball a
             left join Batsman_Scored b
             on a.Match_Id=b.Match_Id 
             and a.Innings_No=b.Innings_No
             and a.Over_Id=b.Over_Id 
             and a.Ball_Id=b.Ball_Id
             left join Extra_Runs c
             on a.Match_Id=c.Match_Id 
             and a.Innings_No=c.Innings_No
             and a.Over_Id=c.Over_Id 
             and a.Ball_Id=c.Ball_Id
             where Extra_Type_Id !=2 or Extra_Type_Id is null
             group by Striker) a
        join(select Player_Out,count(*) as dismissals
             from Wicket_Taken
             group by Player_Out) b
        on a.Striker=b.Player_Out
        join( select Striker,count (distinct Match_Id)as Matches
              from Ball_by_Ball
              group by Striker) c
        on a.Striker=c.Striker
        join (select Player_Id,Player_Name,b.Country_Name, c.Batting_hand
              from Player a
              join Country b
              on a.Country_Name=b.Country_Id
              join Batting_Style c
              on a.Batting_hand=c.Batting_Id) d
        on a.Striker=d.Player_Id
        where Matches>=50
        order by Batting_Avg desc,Strike_Rate desc ; """

highest_avg=pd.read_sql(sql, conn)
subset=["Runs","Batting_Avg","Strike_Rate"]
highest_avg.head(5).style.background_gradient(subset=subset,low=0.55)

,Player,Matches,Runs,Batting_Avg,Strike_Rate
0,CH Gayle,West Indies,3447,43.090000,153.470000
1,SE Marsh,Australia,2225,40.450000,132.680000
2,JP Duminy,South Africa,1993,39.860000,124.870000
3,AB de Villiers,South Africa,3270,39.400000,149.660000
4,MS Dhoni,India,3270,39.400000,138.910000


In [25]:
#Batsman's with lowest strike rate and batting_average in IPL

subset=["Runs","Batting_Avg","Strike_Rate"]
lowest_avg=highest_avg.tail().iloc[::-1,:].reset_index(drop=True)
lowest_avg.style.background_gradient(subset=subset,low=0.55)

,Player,Matches,Runs,Batting_Avg,Strike_Rate
0,P Kumar,India,332,8.970000,108.850000
1,PP Chawla,India,513,12.510000,114.770000
2,Harbhajan Singh,India,783,15.980000,143.670000
3,AM Nayar,India,672,17.680000,116.460000
4,NV Ojha,India,1474,21.060000,118.490000


In [26]:
# Best Batsman's in IPL

sql=""" select Player_Name as Player,
        Matches,Runs,Strike_Rate,
        round(round(Runs,2)/round(dismissals,2),2) as Batting_Avg,
        fifties,centuries,Best_Score
        from(select Striker,sum(coalesce(Runs_Scored,0)) as Runs,
             100*round(round(sum(coalesce(Runs_Scored,0)),2)/round(count(*),2),4) as Strike_Rate
             from Ball_by_Ball a
             left join Batsman_Scored b
             on a.Match_Id=b.Match_Id 
             and a.Innings_No=b.Innings_No
             and a.Over_Id=b.Over_Id 
             and a.Ball_Id=b.Ball_Id
             left join Extra_Runs c
             on a.Match_Id=c.Match_Id 
             and a.Innings_No=c.Innings_No
             and a.Over_Id=c.Over_Id 
             and a.Ball_Id=c.Ball_Id
             where Extra_Type_Id !=2 or Extra_Type_Id is null
             group by Striker) a
        join(select Player_Out,count(*) as dismissals
             from Wicket_Taken
             group by Player_Out) b
        on a.Striker=b.Player_Out
        join( select Striker,sum(case when runs==0 then 1 else 0 end) as Ducks,
              sum(case when runs>=50 and runs<100 then 1 else 0 end) as fifties,
              sum(case when runs>=100 then 1 else 0 end) as centuries,
              max(runs) as Best_Score
              from(select Striker,sum(Runs_Scored) as runs
                   from Ball_by_Ball a
                   join Batsman_Scored b
                   on a.Match_Id=b.Match_Id 
                   and a.Innings_No=b.Innings_No
                   and a.Over_Id=b.Over_Id 
                   and a.Ball_Id=b.Ball_Id
                   group by Striker,a.Match_Id)
               group by Striker) c
        on a.Striker=c.Striker
        join(select Striker,
             sum(case when Runs_Scored==4 then 1 else 0 end) as Fours,
             sum(case when Runs_Scored==6 then 1 else 0 end) as Sixes
             from Ball_by_Ball a
             join Batsman_Scored b
             on a.Match_Id=b.Match_Id 
             and a.Innings_No=b.Innings_No
             and a.Over_Id=b.Over_Id 
             and a.Ball_Id=b.Ball_Id
             group by Striker) d
        on a.Striker=d.Striker
        join( select Striker,count (distinct Match_Id)as Matches
              from Ball_by_Ball
              group by Striker) e
        on a.Striker=e.Striker
        join (select Player_Id,Player_Name,b.Country_Name, c.Batting_hand
              from Player a
              join Country b
              on a.Country_Name=b.Country_Id
              join Batting_Style c
              on a.Batting_hand=c.Batting_Id) f
        on a.Striker=f.Player_Id
        where Matches>=50
        order by Batting_Avg desc,Strike_Rate desc; """
            
best_batters=pd.read_sql(sql, conn)
subset=["Runs","Batting_Avg","Strike_Rate"]
best_batters.head().style.background_gradient(subset=subset,low=0.55)

,Player,Matches,Runs,Strike_Rate,Batting_Avg,fifties,centuries,Best_Score
0,CH Gayle,91,3447,153.470000,43.090000,21,5,175
1,SE Marsh,61,2225,132.680000,40.450000,18,1,115
2,JP Duminy,73,1993,124.870000,39.860000,14,0,78
3,AB de Villiers,109,3270,149.660000,39.400000,22,3,133
4,MS Dhoni,128,3270,138.910000,39.400000,16,0,70


### Fielding Statistics

In [27]:
#Best Fielders in IPL

sql=""" select Player_Name,Country_Name,dismissals,catch,run_out
        from(select Player_Name,Country_Name,count(Kind_Out) as dismissals,
             sum(case when Kind_Out==1 then 1 else 0 end) as catch,
             sum(case when Kind_Out==3 then 1 else 0 end) as run_out,
             sum(case when Kind_Out==6 then 1 else 0 end) as stumping
             from Wicket_Taken a
             join (select Player_Id,Player_Name,b.Country_Name
                   from Player a
                   join Country b
                   on a.Country_Name=b.Country_Id) b
             on a.Fielders=b.Player_Id
             where Kind_Out in (1,3,6)
             group by b.Player_Id
             having stumping=0
             order by dismissals desc) ; """

fielders=pd.read_sql(sql, conn)
fielders.head().style.background_gradient(subset=['dismissals',"catch","run_out"],cmap="Greens",low=0.35)

,Player_Name,Country_Name,dismissals,catch,run_out
0,SK Raina,India,91,79,12
1,RG Sharma,India,69,60,9
2,V Kohli,India,68,55,13
3,DJ Bravo,West Indies,64,52,12
4,KA Pollard,West Indies,57,49,8


In [28]:
#Best Wicket-Keepers in IPL

sql=""" select Player_Name,Country_Name,count(Kind_Out) as dismissals,
        sum(case when Kind_Out==1 then 1 else 0 end) as catch,
        sum(case when Kind_Out==3 then 1 else 0 end) as run_out,
        sum(case when Kind_Out==6 then 1 else 0 end) as stumping
        from Wicket_Taken a
        join (select Player_Id,Player_Name,b.Country_Name
              from Player a
              join Country b
              on a.Country_Name=b.Country_Id) b
        on a.Fielders=b.Player_Id
        where Kind_Out in (1,3,6)
        group by b.Player_Id
        having stumping!=0
        order by dismissals desc; """

keepers=pd.read_sql(sql, conn)
keepers.head().style.background_gradient(subset=['dismissals',"catch","run_out","stumping"],cmap="Greens",low=0.35)

,Player_Name,Country_Name,dismissals,catch,run_out,stumping
0,KD Karthik,India,119,80,13,26
1,MS Dhoni,India,111,66,18,27
2,RV Uthappa,India,100,66,8,26
3,AB de Villiers,South Africa,97,77,13,7
4,NV Ojha,India,76,59,7,10


### Captain Statistics

In [29]:
# Most successful captains of IPL

sql="""select Player_Name as Captain,Country_Name,
       count(*) as Matches,
       sum(case when a.Team_Id=b.Winner then 1 else 0 end) as Wins,
       round(round(sum(case when a.Team_Id=b.Winner then 1 else 0 end),2)/round(count(*),2),2) as Win_perc,
       round(round(sum(case when a.Team_Id=b.Winner then chasing else 0 end),2)/round(sum(case when a.Team_Id!=b.Winner then defending else 0 end+case when a.Team_Id=b.Winner then chasing else 0 end),2),2) as Chasing_perc ,
       round(round(sum(case when a.Team_Id=b.Winner then defending else 0 end),2)/round(sum(case when a.Team_Id!=b.Winner then chasing else 0 end+case when a.Team_Id=b.Winner then defending else 0 end),2),2) as Defending_perc
       from (select Match_Id,Team_Id,Player_Id
             from Player_Match 
             where Role_Id in (1,4)
             group by Match_Id,Team_Id,Player_Id) a
       left join(select Match_Id, 
                 case when Win_Type==1 
                      then case when Toss_Winner=Team_1 and Toss_Decide=2 then Team_1
                                when Toss_Winner=Team_1 and Toss_Decide=1 then Team_2
                                when Toss_Winner=Team_2 and Toss_Decide=2 then Team_2
                                else Team_1 end
                      else case when Toss_Winner=Team_1 and Toss_Decide=2 then Team_2
                                when Toss_Winner=Team_1 and Toss_Decide=1 then Team_1
                                when Toss_Winner=Team_2 and Toss_Decide=2 then Team_1
                                else Team_2 end 
                      end as Winner,
                case when Win_Type==1 then 1 else 0 end as defending,
                case when Win_Type==2 then 1 else 0 end as chasing
                from Match a
                where Win_Type in (1,2)) as b
        on a.Match_Id=b.Match_Id
        join( select Player_Id,Player_Name,b.Country_Name
              from Player a
              join Country b
              on a.Country_Name=b.Country_Id) c
        on a.Player_Id=c.Player_Id
        group by a.Player_Id
        having Matches>=30
        order by Win_perc desc, Chasing_perc desc,Defending_perc desc; """

best_captain=pd.read_sql(sql, conn)
subset=["Win_perc","Defending_perc","Chasing_perc"]
best_captain.head(10).style.background_gradient(subset=subset,cmap="Purples",low=0.35)

,Captain,Country_Name,Matches,Wins,Win_perc,Chasing_perc,Defending_perc
0,RG Sharma,India,58,34,0.590000,0.640000,0.550000
1,SR Tendulkar,India,51,30,0.590000,0.550000,0.620000
2,DA Warner,Australia,33,19,0.580000,0.470000,0.670000
3,MS Dhoni,India,142,81,0.570000,0.600000,0.550000
4,G Gambhir,India,107,60,0.560000,0.590000,0.520000
5,V Sehwag,India,53,29,0.550000,0.660000,0.430000
6,SK Warne,Australia,55,30,0.550000,0.650000,0.430000
7,V Kohli,India,71,36,0.510000,0.610000,0.470000
8,Yuvraj Singh,India,43,21,0.490000,0.570000,0.420000
9,AC Gilchrist,Australia,74,35,0.470000,0.460000,0.490000


### Team Statistics

In [30]:
# Best Teams in IPL

sql=""" select Team_Name as franchise,
        a.matches+b.matches as Matches,c.Wins,
        round(round(Wins,2)/round(a.matches+b.matches,2),2) as Win_perc
        from( select Season_Id,Team_1,
              count(*) as matches
              from Match
              group by Team_1) a
        join( select Season_Id,Team_2,
              count(*) as matches
              from Match
              group by Team_2) b
        on a.Team_1=b.Team_2
        join (select Match_Winner,
              count(*) as Wins
              from Match
              where Match_Winner is not null
              group by Match_Winner) c
        on a.Team_1=c.Match_Winner
        join Team d
        on a.Team_1=d.Team_Id
        order by Win_perc desc; """

best_teams=pd.read_sql(sql, conn).set_index("franchise")
best_teams.style.background_gradient(subset=["Wins","Win_perc"],cmap="Oranges",high=0.5)

,Matches,Wins,Win_perc
franchise,,,
Chennai Super Kings,131,79,0.600000
Mumbai Indians,140,80,0.570000
Gujarat Lions,16,9,0.560000
Sunrisers Hyderabad,62,34,0.550000
Rajasthan Royals,118,63,0.530000
Kolkata Knight Riders,132,68,0.520000
Royal Challengers Bangalore,139,70,0.500000
Kings XI Punjab,134,63,0.470000
Kochi Tuskers Kerala,14,6,0.430000


In [31]:
# best team in defending the score

sql=""" select 
        case when Toss_Winner=Team_1 and Toss_Decide=2 then b.Team_Name
             when Toss_Winner=Team_1 and Toss_Decide=1 then c.Team_Name
             when Toss_Winner=Team_2 and Toss_Decide=2 then c.Team_Name
             else b.Team_Name end as franchise,
        count(Match_Id) as matches,sum(case when Win_Type==1 then 1 else 0 end)as wins,
        round(round(sum(case when Win_Type==1 then 1 else 0 end),2)/round(count(Match_Id),2),2) as win_percentage
        from Match a
        join Team b
        on a.Team_1=b.Team_Id
        join Team c
        on a.Team_2=c.Team_Id
        group by franchise
        order by win_percentage desc; """

best_defenders=pd.read_sql(sql, conn).set_index("franchise")
best_defenders.style.background_gradient(subset=["win_percentage"],cmap="Oranges",high=0.5)

,matches,wins,win_percentage
franchise,,,
Chennai Super Kings,77,46,0.600000
Mumbai Indians,76,43,0.570000
Sunrisers Hyderabad,37,19,0.510000
Kolkata Knight Riders,62,28,0.450000
Royal Challengers Bangalore,62,27,0.440000
Rajasthan Royals,54,23,0.430000
Deccan Chargers,43,18,0.420000
Kings XI Punjab,69,28,0.410000
Delhi Daredevils,57,20,0.350000


In [32]:
# best team in chasing the score

sql=""" select 
        case when Toss_Winner=Team_1 and Toss_Decide=2 then c.Team_Name
             when Toss_Winner=Team_1 and Toss_Decide=1 then b.Team_Name
             when Toss_Winner=Team_2 and Toss_Decide=2 then b.Team_Name
             else c.Team_Name end as franchise,
        count(Match_Id) as matches,sum(case when Win_Type==2 then 1 else 0 end)as wins,
        round(round(sum(case when Win_Type==2 then 1 else 0 end),2)/round(count(Match_Id),2),2) as win_percentage
        from Match a
        join Team b
        on a.Team_1=b.Team_Id
        join Team c
        on a.Team_2=c.Team_Id
        group by franchise
        order by win_percentage desc; """

best_chasers=pd.read_sql(sql, conn).set_index("franchise")
best_chasers.style.background_gradient(subset=["win_percentage"],cmap="Oranges",high=0.5)

,matches,wins,win_percentage
franchise,,,
Gujarat Lions,10,8,0.800000
Chennai Super Kings,54,33,0.610000
Sunrisers Hyderabad,25,15,0.600000
Rajasthan Royals,64,38,0.590000
Mumbai Indians,64,37,0.580000
Kochi Tuskers Kerala,7,4,0.570000
Kolkata Knight Riders,70,39,0.560000
Royal Challengers Bangalore,77,42,0.550000
Kings XI Punjab,65,33,0.510000


In [33]:
# Best Teams in IPL

sql=""" select Team_Name as Team,
        a.matches+b.matches as Matches,c.Wins,
        round(round(Wins,2)/round(a.matches+b.matches,2),2) as Win_perc,
        Defending_perc,Chasing_perc
        from( select Team_1,
              count(*) as matches
              from Match
              group by Team_1) a
        join( select Team_2,
              count(*) as matches
              from Match
              group by Team_2) b
        on a.Team_1=b.Team_2
        join (select Match_Winner,
              count(*) as Wins
              from Match
              where Match_Winner is not null
              group by Match_Winner) c
        on a.Team_1=c.Match_Winner
        join( select 
              case when Toss_Winner=Team_1 and Toss_Decide=2 then Team_1
                   when Toss_Winner=Team_1 and Toss_Decide=1 then Team_2
                   when Toss_Winner=Team_2 and Toss_Decide=2 then Team_2
                   else Team_1 end as franchise,
              round(round(sum(case when Win_Type==1 then 1 else 0 end),2)/round(count(Match_Id),2),2) as Defending_perc
              from Match 
              where Outcome_type=1 
              group by franchise) d
        on a.Team_1=d.franchise
        join( select 
              case when Toss_Winner=Team_1 and Toss_Decide=2 then Team_2
                   when Toss_Winner=Team_1 and Toss_Decide=1 then Team_1
                   when Toss_Winner=Team_2 and Toss_Decide=2 then Team_1
                   else Team_2 end as franchise,
              round(round(sum(case when Win_Type==2 then 1 else 0 end),2)/round(count(Match_Id),2),2) as Chasing_perc
              from Match 
              where Outcome_type=1
              group by franchise) e
        on a.Team_1=e.franchise
        join Team f
        on a.Team_1=f.Team_Id
        order by Win_perc desc,Chasing_perc desc,Defending_perc desc; """

best_teams=pd.read_sql(sql, conn)
subset=["Win_perc","Defending_perc","Chasing_perc"]
best_teams.style.background_gradient(subset=subset,cmap="Oranges",high=0.5)

,Team,Matches,Wins,Win_perc,Defending_perc,Chasing_perc
0,Chennai Super Kings,131,79,0.600000,0.600000,0.620000
1,Mumbai Indians,140,80,0.570000,0.570000,0.580000
2,Gujarat Lions,16,9,0.560000,0.170000,0.800000
3,Sunrisers Hyderabad,62,34,0.550000,0.510000,0.630000
4,Rajasthan Royals,118,63,0.530000,0.450000,0.600000
5,Kolkata Knight Riders,132,68,0.520000,0.450000,0.570000
6,Royal Challengers Bangalore,139,70,0.500000,0.450000,0.560000
7,Kings XI Punjab,134,63,0.470000,0.410000,0.520000
8,Kochi Tuskers Kerala,14,6,0.430000,0.290000,0.570000
9,Delhi Daredevils,133,56,0.420000,0.370000,0.500000


### Season Statistics

In [34]:
# IPL Season's Best PLayers

sql=""" select Season_Year,
        b.Player_Name as Man_of_Season,
        c.Player_Name as Top_Scorer,
        d.Player_Name as Top_Wicket_Tacker
        from Season a
        join Player b
        on a.Man_of_the_Series=b.Player_Id
        join Player c
        on a.Orange_Cap=c.Player_Id
        join Player d
        on a.Purple_Cap=d.Player_Id; """

top_perfomer=pd.read_sql(sql, conn).set_index("Season_Year")
top_perfomer

,Man_of_Season,Top_Scorer,Top_Wicket_Tacker
Season_Year,,,
2008,SR Watson,SE Marsh,Sohail Tanvir
2009,AC Gilchrist,ML Hayden,RP Singh
2010,SR Tendulkar,SR Tendulkar,PP Ojha
2011,CH Gayle,CH Gayle,SL Malinga
2012,SP Narine,CH Gayle,M Morkel
2013,SR Watson,MEK Hussey,DJ Bravo
2014,GJ Maxwell,RV Uthappa,MM Sharma
2015,AD Russell,DA Warner,DJ Bravo
2016,V Kohli,V Kohli,B Kumar


In [35]:
# IPL Season's Winners,Runners Up, Win Type

sql=""" select Season_Year,c.Team_Name as Winner,d.Team_Name as Runner_Up
        from( select Season_Id,
              case when Team_1=Match_Winner then Team_1 
              else Team_2 end as first,
              case when Team_1!=Match_Winner then Team_1 
              else Team_2 end as second
              from (select Season_Id,Team_1,Team_2,Match_Winner
                    from Match
                    group by Season_Id
                    having Match_Id=max(Match_Id))) a
        join Season b
        on a.Season_Id=b.Season_Id
        join Team c
        on a.first=c.Team_Id
        join Team d
        on a.second=d.Team_Id; """

season_winner=pd.read_sql(sql, conn).set_index("Season_Year")
season_winner

,Winner,Runner_Up
Season_Year,,
2008,Rajasthan Royals,Chennai Super Kings
2009,Deccan Chargers,Royal Challengers Bangalore
2010,Chennai Super Kings,Mumbai Indians
2011,Chennai Super Kings,Royal Challengers Bangalore
2012,Kolkata Knight Riders,Chennai Super Kings
2013,Mumbai Indians,Chennai Super Kings
2014,Kolkata Knight Riders,Kings XI Punjab
2015,Mumbai Indians,Chennai Super Kings
2016,Sunrisers Hyderabad,Royal Challengers Bangalore


In [36]:
# Runs scored in powerplay,middle and death overs in different seasons of IPL

sql=""" select Season_Year,Matches,
        sum(case when a.Over_Id<=6 then Runs_Scored+coalesce(Extra_Runs,0) else 0 end) as powerplay,
        sum(case when a.Over_Id>6 and a.Over_Id<=15 then Runs_Scored+coalesce(Extra_Runs,0) else 0 end) as middleovers,
        sum(case when a.Over_Id>15 then Runs_Scored+coalesce(Extra_Runs,0) else 0 end) as deathovers
        from Batsman_Scored a
        join Match b
        on a.Match_Id==b.Match_Id
        left join Extra_Runs c
        on a.Match_Id=c.Match_Id 
        and a.Innings_No=c.Innings_No
        and a.Over_Id=c.Over_Id
        and a.Ball_Id=c.Ball_Id
        join Season d
        on b.Season_Id==d.Season_Id
        join (select Season_Id,
              count(Match_Id) as Matches
              from Match 
              group by Season_Id) e
        on b.Season_Id==e.Season_Id
        group by d.Season_Year; """

runs_season=pd.read_sql(sql, conn).set_index("Season_Year")
subset=["powerplay","middleovers","deathovers"]
runs_season.head().style.background_gradient(subset=subset,cmap="YlOrRd",high=0.5)

,Matches,powerplay,middleovers,deathovers
Season_Year,,,,
2008,58,5111,7821,4400
2009,57,4749,6695,4382
2010,60,5445,7860,4939
2011,73,5998,9236,5264
2012,74,6404,9771,6252


In [37]:
#highest score of a Season 

sql=""" select Season_Year,
        case when a.Innings_No==1 then c.Team_Name 
             else d.Team_Name end as batting_team,
        case when a.Innings_No==1 then d.Team_Name 
             else c.Team_Name end as fielding_team,
        max(runs+extra) as Score,City_Name
        from(select Season_Year,sum(Runs_Scored) as runs,
             a.Match_Id,a.Innings_No,Venue_id,
             case when Toss_Winner==Team_1 and Toss_Decide==1 then Team_2
                  when Toss_Winner==Team_1 and Toss_Decide==2 then Team_1
                  when Toss_Winner==Team_2 and Toss_Decide==1 then Team_1
                  else Team_2 end as batting,
             case when Toss_Winner==Team_1 and Toss_Decide==1 then Team_1
                  when Toss_Winner==Team_1 and Toss_Decide==2 then Team_2
                  when Toss_Winner==Team_2 and Toss_Decide==1 then Team_2
                  else Team_1 end as fielding
            from Batsman_Scored a
            join Match b
            on a.Match_Id=b.Match_Id
            join Season d
            on b.Season_Id=d.Season_Id
            group by Season_Year, a.Match_Id, a.Innings_No) a
        join (select Match_Id,Innings_No,sum(Extra_Runs) as extra
                  from Extra_Runs
                  group by Match_Id,Innings_No) b
        on a.Match_Id=b.Match_Id 
        and a.Innings_No=b.Innings_No
        join Team c
        on a.batting=c.Team_Id
        join Team d
        on a.fielding=d.Team_Id
        join (Select Venue_Id,Venue_Name,City_Name
              from Venue a
              join City b
              on a.City_Id=b.City_Id) e
        on a.Venue_Id=e.Venue_Id
        group by Season_Year; """

Season_highest=pd.read_sql(sql, conn).set_index("Season_Year")
Season_highest.head().style.background_gradient(subset=["Score"],cmap="YlOrRd",high=0.5)

,batting_team,fielding_team,Score,City_Name
Season_Year,,,,
2008,Chennai Super Kings,Kings XI Punjab,240,Chandigarh
2009,Rajasthan Royals,Kings XI Punjab,211,Durban
2010,Chennai Super Kings,Rajasthan Royals,246,Chennai
2011,Kings XI Punjab,Royal Challengers Bangalore,232,Dharamsala
2012,Chennai Super Kings,Delhi Daredevils,222,Chennai


In [38]:
#Lowest score of a Season 

sql=""" select Season_Year,
        case when a.Innings_No==1 then c.Team_Name 
             else d.Team_Name end as batting_team,
        case when a.Innings_No==1 then d.Team_Name 
             else c.Team_Name end as fielding_team,
        min(runs+extra) as Score,City_Name
        from(select Season_Year,sum(Runs_Scored) as runs,
             a.Match_Id,a.Innings_No,Venue_id,
             case when Toss_Winner==Team_1 and Toss_Decide==1 then Team_2
                  when Toss_Winner==Team_1 and Toss_Decide==2 then Team_1
                  when Toss_Winner==Team_2 and Toss_Decide==1 then Team_1
                  else Team_2 end as batting,
             case when Toss_Winner==Team_1 and Toss_Decide==1 then Team_1
                  when Toss_Winner==Team_1 and Toss_Decide==2 then Team_2
                  when Toss_Winner==Team_2 and Toss_Decide==1 then Team_2
                  else Team_1 end as fielding
            from Batsman_Scored a
            join Match b
            on a.Match_Id=b.Match_Id
            join Season d
            on b.Season_Id=d.Season_Id
            where Win_Type not in (3,4)
            group by Season_Year, a.Match_Id, a.Innings_No) a
        join (select Match_Id,Innings_No,sum(Extra_Runs) as extra
                  from Extra_Runs
                  group by Match_Id,Innings_No) b
        on a.Match_Id=b.Match_Id 
        and a.Innings_No=b.Innings_No
        join Team c
        on a.batting=c.Team_Id
        join Team d
        on a.fielding=d.Team_Id
        join (Select Venue_Id,Venue_Name,City_Name
              from Venue a
              join City b
              on a.City_Id=b.City_Id) e
        on a.Venue_Id=e.Venue_Id
        group by Season_Year; """

Season_lowest=pd.read_sql(sql, conn).set_index("Season_Year")
Season_lowest.head().style.background_gradient(subset=["Score"],cmap="YlOrRd",high=0.5)

,batting_team,fielding_team,Score,City_Name
Season_Year,,,,
2008,Chennai Super Kings,Kolkata Knight Riders,55,Kolkata
2009,Rajasthan Royals,Royal Challengers Bangalore,58,Cape Town
2010,Deccan Chargers,Royal Challengers Bangalore,82,Mumbai
2011,Kolkata Knight Riders,Chennai Super Kings,61,Kolkata
2012,Mumbai Indians,Delhi Daredevils,92,Mumbai


In [39]:
# Season Wise Team Performance

sql=""" select Season_Year,Team_Name,
        a.matches+b.matches as Matches,c.Wins,
        round(round(Wins,2)/round(a.matches+b.matches,2),2) as Win_perc,
        Defending_perc,Chasing_perc
        from( select Season_Id,Team_1,
              count(*) as matches
              from Match
              group by Season_Id,Team_1) a
        join( select Season_Id,Team_2,
              count(*) as matches
              from Match
              group by Season_Id,Team_2) b
        on a.Season_Id=b.Season_Id
        and a.Team_1=b.Team_2
        join (select Season_Id,Match_Winner,
              count(*) as Wins
              from Match
              where Match_Winner is not null
              group by Season_Id,Match_Winner) c
        on a.Season_Id=c.Season_Id
        and a.Team_1=c.Match_Winner
        join( select Season_Id,
              case when Toss_Winner=Team_1 and Toss_Decide=2 then Team_1
                   when Toss_Winner=Team_1 and Toss_Decide=1 then Team_2
                   when Toss_Winner=Team_2 and Toss_Decide=2 then Team_2
                   else Team_1 end as franchise,
              round(round(sum(case when Win_Type==1 then 1 else 0 end),2)/round(count(Match_Id),2),2) as Defending_perc
              from Match 
              where Outcome_type=1 
              group by Season_Id,franchise) d
        on a.Season_Id=d.Season_Id
        and a.Team_1=d.franchise
        join( select Season_Id,
              case when Toss_Winner=Team_1 and Toss_Decide=2 then Team_2
                   when Toss_Winner=Team_1 and Toss_Decide=1 then Team_1
                   when Toss_Winner=Team_2 and Toss_Decide=2 then Team_1
                   else Team_2 end as franchise,
              round(round(sum(case when Win_Type==2 then 1 else 0 end),2)/round(count(Match_Id),2),2) as Chasing_perc
              from Match 
              where Outcome_type=1
              group by Season_Id,franchise) e
        on a.Season_Id=e.Season_Id
        and a.Team_1=e.franchise
        join Team f
        on a.Team_1=f.Team_Id
        join Season g
        on a.Season_Id=g.Season_Id
        order by a.Season_Id,Matches desc,Wins desc,Chasing_perc desc; """

beta=pd.read_sql(sql, conn).set_index(keys=["Season_Year","Team_Name"])

# Select Season year to display, default=2016
subset=["Wins","Win_perc","Defending_perc","Chasing_perc"]
beta.loc[2016].style.background_gradient(subset=subset,cmap="YlOrRd",high=0.5)

,Matches,Wins,Win_perc,Defending_perc,Chasing_perc
Team_Name,,,,,
Sunrisers Hyderabad,17,11,0.650000,0.670000,0.750000
Gujarat Lions,16,9,0.560000,0.170000,0.800000
Royal Challengers Bangalore,16,9,0.560000,0.500000,0.670000
Kolkata Knight Riders,15,8,0.530000,0.290000,0.750000
Mumbai Indians,14,7,0.500000,0.290000,0.710000
Delhi Daredevils,14,7,0.500000,0.500000,0.630000
Rising Pune Supergiants,14,5,0.360000,0.000000,0.430000
Kings XI Punjab,14,4,0.290000,0.250000,0.330000
